# Speeding up your code through vectorizing

As we said in the introduction, `python` isn't a compiled programming language like `C`, `java`, or `FORTRAN`. While `python` makes it much easier to glue together many different libraries to do some amazing things, its achiles heal is that it is still "just" a scripting language. If you try to write a `python` program for numerical computation like you would a `C` program, with explicit loops, your code will be slow to run. This notebook demonstrates this and shows you some tips and tricks to "vectorize" your code. Always ask yourself:  can I replace this block of code with calls to `numpy` or other such optimized libraries?

## Loops are deadly

In astronomy, we work quite a bit with 2D arrays, like CCD images. Or catalogs with millions of entries. Trying to do math on these with `for` loops will really slow you down. Here's an example. We'll load up some `FITS` data from the visualization notebook and do a common task:  remove the background sky level, which we estimate from the median of all the pixels.

In [ ]:
from astropy.io import fits
import numpy as np
fts = fits.open('../data/SN2011iv_B_SWO_DC_2011_12_11SN.fits')
print(fts[0].data.shape)
data = fts[0].data
sky_level = np.median(data)
print("Approximate sky level = {}".format(sky_level))

Now we do the computation two different ways. First, we loop through all the pixels of the image using nested for-loops. The "magic" command `%%time` will time how long it takes the entire cell to run.

In [ ]:
%%time
sub = np.zeros(data.shape)
for j in range(data.shape[0]):
    for i in range(data.shape[1]):
        sub[j,i] = data[j,i] - sky_level

Depending on your hardware, this will take different amounts of time to run. For something reasonably new (post 2015), you'd expect something like a second or less. Now let's do the same thing but use the fact that, being a `numpy` array, we can simply do math on it in one line without a loop.

In [ ]:
%time sub = data - sky_level

The result will likely be in milli-seconds rather than seconds. You'll probably see something like a factor of 100 improvement. That factor will get larger and larger as the data sizes get larger and larger. By the way, what's happening in the previous cell is `python` sees the subtraction operator, so looks up the `data` object's `__sub__()` function and calls it with the value of `sky_level`. The `__sub__()` function uses pre-compiled `C` code to do the math, which is much faster than a `python` loop. This is true of all the `numpy` math functions and array manipulation functions, so the more you can write your program in terms of these functions rather than loops, the faster your code will run.

## numpy.indices

In the previous example, we did math on a 2D image and looped over the index of the pixels `[j,i]` (incidentally, the reason I use `[j,i]` rather than `[i,j]` is that in `python` what we normally think of as the horizontal axis is the *second* index in `numpy` arrays). The math in question, subtracting a constant value, didn't depend on the actual index values themselves, so translating to a simple `numpy` "one-liner" was easy. But let's say we needed to do something to an array that depends on the index position. As a completely made-up example, let's say we have to subtract `sky_level` from each pixel multiplied by the distance of the pixel from the center of the image. How do we do this with `numpy` functions? Well, first, we can create another 2D array whose values are the pixel indices themselves:

In [ ]:
jj,ii = np.indices(data.shape)
print(jj)
print(ii)
print(ii[101,10],jj[101,10])

Take a second to look at `jj` and ``ii`` and understand what they are: a way to refer to the *index value* of an associted array. We can now formulate more complicated expressions that need the index values. So for the made-up example, we now need a way to refer to the distance of each pixel from the center of the image. We can make *another* array, which has this information. We'll then plot the 3 arrays as images so you can visualize them.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
# center of the 2D image
j0 = data.shape[0]//2
i0 = data.shape[1]//2
# distances of each pixel from that center
dists = np.sqrt((jj - j0)**2 + (ii - i0)**2)
print(dists)
# The math we wanted to do:
bogus = data - dists*sky_level

# make images#
fig,axes = plt.subplots(1,4, figsize=(12,5))
axes[0].imshow(ii, origin='lower')
axes[1].imshow(jj, origin='lower')
axes[2].imshow(dists, origin='lower')
axes[3].imshow(bogus, origin='lower', vmin=-100, vmax=100)

## numpy.newaxis

This one's a super handy trick, but can be hard to get your head around at first. In the previous examples, we worked with math on arrays with consistent shapes (2D images with same numbers of pixels in both dimensions). But sometimes you want to do math on arrays that have different dimensions. Building on the previous example of estimating a background and removing it, let's look at spectroscopy instead. You still have a 2D image, but in this case, the horizontal (2nd index) represents wavelength, while the vertical axis represents "distance along the slit" of the spectrograph. Let's plot up a sample spectrum.

In [ ]:
spec = fits.open('data/2D_spectrum.fts')
fig,ax = plt.subplots(figsize=(10,5))
ax.imshow(spec[0].data, origin='lower', vmin=900, vmax=1000)

What stands out in the spetrum are the bright vertical lines. This is emission at specific wavelengths and are caused by our atmosphere. Unlike our previous example, where we just computed a single value for the sky level, here it is a strong function of wavelength. But we can still remove it by computing the median of the image, but only along the first index (which is the vertical direction). Let's do that and plot it.

In [ ]:
sky_level = np.median(spec[0].data, axis=0)   # axis=0 means:  take the median along this axis
plt.plot(sky_level)
plt.xlabel('pixel wavelength')
plt.ylabel('sky level (counts)')

So now we have a 2D image, and a 1D estimate of the sky background. We'd like to subtract `sky_level` from the data, but that would raise an exception because the two arrays have different shapes. We could just loop over each row of the data and subtract the same sky, but we want to avoid loops. 

This is where `newaxis` comes in. You can replace a "missing" dimension with `newaxis` and `numpy` will fill in that missing dimension by making copies. In this case, the "missing" dimension in `sky_level` is the vertical one in the data, which for `numpy` is the first index. So we pretend `sky_level` has two indexes, and specify `np.newaxis` for the first.

In [ ]:
print(sky_level.shape)
subtracted = spec[0].data - sky_level
fig,ax = plt.subplots(figsize=(10,5))
ax.imshow(subtracted, origin='lower', vmin=-50, vmax=50)

Much better! You might notice that the lines don't subtract out perfectly, especially at the edges of the frame. That's because the lines are not perfectly straight (the dispersing element of the spectrograph is not pefectly aligned with the CCD). Incidentally, this is a spectrum of a galaxy (NGC 1313) and the "blobs" are emission from HII regions. There are also some cosmic-rays (high-energy particles) that appear as sharp points that would need to be removed.

## Outer Joins with newaxis

Another trick with `newaxis` is to do math on 2 1D arrays and create a higher dimensional object. A classic example of this is when you have two separate catalogs of objects with coordinates on the sky, but no names or labels you can use to match them up. You want to find objects that are "close" to each other (of course you have to decide what "close" means). For example, we include source catalogs around the supernova 2021csp from an image taken at the Swope telescope (using the program [Source Extractor](https://sextractor.readthedocs.io/en/latest/). We want to match these objects we got from querying the [PanSTARRs](https://panstarrs.stsci.edu/) catalog. We can read both of them in with `astropy`.

In [ ]:
from astropy.io import ascii
tab1 = ascii.read('data/2021csp_se.cat')
tab2 = ascii.read('data/2021csp_PS.cat')
print(tab1)
print(tab2)

As you can see, both tables have an `RA` and `DEC`. Because of measurement error, propoer motions, etc, the same object won't have exactly the same coordinates in both catalogs. So we could loop over every object in `tab1` and construct distances from that object to every other object in the `tab2`, find the minimum, and decide if it's "close enough" to make a match. To avoid a loop, we can construct a 2D array `distances`. We'll make it so that `distance[i,j]` is the distance between object `tab1[i]` and object `tab2[j]`. Again, `newaxis` creates a new dimension by making copies, so for tab1 we do this in the second axis and for tab2, we do it in the first. We also have to do a bit of spherical trigonometry which takes into account the fact that as you approach the celestial poles, intervals of RA get smaller, hence the cosine factor.

In [ ]:
deltaRA = tab1['RA'][:,np.newaxis] - tab2['RA'][np.newaxis,:]                    # in degrees
deltaDEC = tab1['DEC'][:,np.newaxis] - tab2['DEC'][np.newaxis,:]                 # in degrees
aveDEC = (tab1['DEC'][:,np.newaxis] + tab2['DEC'][np.newaxis,:])/2*np.pi/180     # in radians
distances = np.sqrt(deltaRA**2*np.cos(aveDEC)**2 + deltaDEC**2)
print(distances.shape)
print(distances)

Lastly, we use `numpy.min` to find the minimum separation for each object in `tab1`. We keep only the ones that are less than 1 arc-second (roughly the uncertainty we expect from both Swope plus PanSTARRs). We then use `np.argmin` to determine the *index* of the object in `tab2` that has the minimum distance to each object in `tab1` and use that to make a re-ordered version of `tab2` that matches the good objects in `tab1`.

In [ ]:
min_distances = np.min(distances, axis=1)*3600     # In arc-seconds
good = min_distances < 1                           # 1 arc-second tolerance
# only keep the objects that have a match
tab1_good = tab1[good]
# The index of tab2 that corresponds to closest object in tab1
tab2_idx = np.argmin(distances, axis=1)
# re-order tab2 to match tab1, then take only the good matches
tab2_good = tab2[tab2_idx][good]
print(tab1_good)
print(tab2_good)

Now, all that work could have been done with `astropy.coordinates`, which has a function for cross-matching tables based on sky coordinates. But we did it this way "by hand" just for illustration.

**A word of Caution**:  When you use `newaxis` and increase the dimensionality of an array, you can very easily end up with something that surpasses your computer's available memory. Just imagine trying to do what we did above with tables that had a million rows each. You would have ended up with matrices that had $(10^6)^2$ entries. Each floating point number is at least 4 bytes, so that would be a total of 3.6 Tera-bytes! Clearly in that case, what we did above is not the approach you want to take. 

## Mandelbrot Set

Hey, why not? If you google how to do the mandlebrot set in python, you get something that uses loops very heavily. Let's do that here and time it. If you don't know what the Mandelbrot set is, there's a good reference [here](https://www.youtube.com/watch?v=FFftmWSzgmk). But we basically look at the complex plane and for each complex value $c$, we feed it into the iterative formula $z_{n+1} = z_n^2 + c$ with $z_0 = 0$. If the norm of $z$ is less than 2, we keep going, but if it gets larger than 2, it will diverge and $c$ is considered to be outside the Mandelbrot set. If we plot the number of iterations before it it diverges to > 2, we get a really cool image. So, first the bad way to do it. We'll make a cell to setup some dimensions first (so you can play around with them).

In [ ]:
NX = 1024     # size of the image in X direction
NY = 512      # size of the image in Y direction
MAXITER = 100 # Maximum number of times we iterate
# Next, the extent of the complex plane we investivate (change these to 'zoom in')
real_min = -2
real_max = 1
imag_min = -1
imag_max = 1

In [ ]:
%%time
# Create an array to count the number until divergence
Ns = np.zeros((NY,NX), dtype=int)

# Make a function that computes N for a given complex number c
def mandelbrot(c):
    z = 0 + 0j       # python uses the engineering notation j = sqrt(-1)
    N = 0
    while abs(z) < 2 and N < MAXITER:
        z = z*z + c
        N += 1
    return N

for j in range(NY):
    img = imag_min + j*(imag_max-imag_min)/NY
    for i in range(NX):
        c = complex(real_min + i*(real_max-real_min)/NX, img)
        Ns[j,i] = mandelbrot(c)
 
fig,ax = plt.subplots(figsize=(15,10))
ax.imshow(Ns, extent=(real_min,real_max,imag_min,imag_max))
ax.set_xlabel('Real(z)')
ax.set_ylabel('Imaginary(z)')

Now we do the same thing, but this time using `numpy` calls instead. Note that we are left with one outer loop, but since the majority of the computation happens inside the loop, it shouldn't impact the performance too much. Plus I couldn't think of a way to vectorize it efficiently.

In [ ]:
%%time
Ns = np.zeros((NY,NX), dtype=int)
imag,real = np.indices((NY,NX))
real = real_min + real/NX*(real_max - real_min)
imag = imag_min + imag/NY*(imag_max - imag_min)
Cs = real + imag*1j
Zs = np.zeros((NY,NX), dtype=complex)
for i in range(100): 
    Zs = Zs**2 + Cs
    Ns = Ns + np.less(np.absolute(Zs),2)

fig,ax = plt.subplots(figsize=(15,10))
ax.imshow(Ns, extent=(real_min,real_max,imag_min,imag_max))
ax.set_xlabel('Real(z)')
ax.set_ylabel('Imaginary(z)')

You will see some warning messages about "overflow". That's because when the $z_n$ go beyond $|z_n| > 2$, they diverve *very* quickly and you end up with numbers larger than `numpy` can handle. That's "okay" and as long as you are aware that some of your array elements may become `nan` (not a number) or `inf` (infinity) and treat them accordingly, such warnings can be ignored. In our case, `np.less(np.nan, 2)` is False, so the code works as intended. But I might have written code that tested `np.greater(np.nan, 2)` and that is also False. In that case, the code would have failed to work as intended. So be careful with `nan` and `inf` values.

We could fix this. Unlike the first Mandelbrot example, which stops iterating when $|z_n| > 2$, in the second example, we compute $z_n$ for *every* $c$, even after some have gone beyond $|z_N| > 2$. So we are actually doing *more* computations than in the first example and still end up ahead. There is a pretty simple way to keep track of those diverging cases and not bother computing their next iterations, avoiding that `nan` and maybe reducing the computations. See if you can figure it out.